In [1]:
# train.py

import os
import torch
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

from data_loader import load_data_1m
from feature_calculations import (
    resample_data, calculate_MA_data, calculate_ema_bollinger_bands, calculate_rsi,
    calculate_macd, calculate_stochastic_oscillator, calculate_adx, calculate_atr,
    calculate_obv, calculate_williams_r, base_feature_fn, cyclic_encode_fn, log_transform
)
from strategies import BB_fitness_fn, BB_MACD_fitness_fn
from dataset import make_dataset, replace_nan_with_zero
from train_functions_rl import inference, fitness_fn, generation_valid, generation_test

from Prescriptor import Prescriptor, ChromosomeSelectorModel
from Evolution.crossover import UniformCrossover, WeightedSumCrossover, DifferentialEvolutionOperator, CenDE_DOBLOperator
from Evolution.mutation import MultiplyNormalMutation, MultiplyUniformMutation, AddNormalMutation, AddUniformMutation, ChainMutation, FlipSignMutation
from Evolution.mutation import RandomValueMutation
from Evolution.selection import RouletteSelection, TournamentSelection, ParetoLexsortSelection
from Evolution import Evolution

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import bisect

class RLDataset(Dataset):
    def __init__(self, data_1m: pd.DataFrame, data_1d: pd.DataFrame, feature_columns: list, close_time_list: list, index_list: list):
        """
        초기화 메서드
        """
        self.data_1m = data_1m
        self.data_1d = data_1d
        self.feature_columns = feature_columns
        self.close_time_list = close_time_list
        self.index_list = sorted(index_list)  # 정렬된 리스트로 가정
        
        # data_1m의 'Close time'을 numpy array로 변환하여 searchsorted 사용
        self.data_1m_times = self.data_1m['Close time'].values
        self.index_array = sorted(self.index_list)  # 이진 탐색을 위해 정렬

    def __len__(self):
        """
        데이터셋의 전체 길이를 반환합니다.
        """
        return len(self.close_time_list)

    def __getitem__(self, idx):
        """
        주어진 인덱스에 해당하는 데이터를 반환합니다.
        """
        # 1. close_time_list에서 해당 인덱스의 날짜를 가져옵니다.
        current_date = self.close_time_list[idx]
        
        # 2. data_1d에서 현재 날짜를 포함한 60일 이전의 데이터를 가져옵니다.
        start_date = current_date - pd.Timedelta(days=59)  # 현재 날짜 포함 60일
        mask = (self.data_1d['Close time'] >= start_date) & (self.data_1d['Close time'] <= current_date)
        data_60d = self.data_1d.loc[mask, self.feature_columns]
        
        # 데이터가 충분하지 않으면 예외 처리
        if len(data_60d) < 60:
            raise ValueError(f"Not enough data for index {idx}: expected 60 days, got {len(data_60d)} days.")
        
        # 피처를 텐서로 변환
        features = torch.tensor(data_60d.values, dtype=torch.float32)
        
        # 3. 다음 날짜의 시작 인덱스를 data_1m에서 찾습니다.
        next_date = current_date + pd.Timedelta(days=1)
        
        # 이진 탐색을 사용하여 next_date 이상의 첫 번째 위치 찾기
        start_pos = bisect.bisect_left(self.data_1m_times, next_date)
        if start_pos >= len(self.data_1m_times):
            raise ValueError(f"No data found in data_1m after {next_date}.")
        start_index = self.data_1m.index[start_pos]
        
        # 4. index_list에서 start_index보다 큰 첫 번째 값을 찾습니다.
        # 이진 탐색을 사용하여 start_index보다 큰 첫 번째 값 찾기
        greater_pos = bisect.bisect_right(self.index_array, start_index)
        if greater_pos >= len(self.index_array):
            raise ValueError(f"No index in index_list greater than start_index {start_index}.")
        first_greater_index = self.index_array[greater_pos]
        
        # 5. 다음 날짜 + 30일의 날짜를 계산하고, 그 날짜의 마지막 인덱스를 찾습니다.
        end_date = current_date + pd.Timedelta(days=31)  # 다음날 포함 30일
        # 이진 탐색을 사용하여 end_date 이하의 마지막 위치 찾기
        end_pos = bisect.bisect_right(self.data_1m_times, end_date) - 1
        if end_pos < 0:
            end_index = self.index_array[-1]
        else:
            end_index_candidate = self.data_1m.index[end_pos]
            # index_list에서 end_index_candidate보다 작은 가장 큰 값 찾기
            less_pos = bisect.bisect_left(self.index_array, end_index_candidate)
            if less_pos == 0:
                raise ValueError(f"No index in index_list less than end_index_candidate {end_index_candidate}.")
            end_index = self.index_array[less_pos - 1]
        
        # 6. 추출한 데이터 프레임, 시작 index, end_index를 반환합니다.
        return features, first_greater_index, end_index, greater_pos, less_pos - 1
    
def data_setting():
    # Load Data
    data_1m = load_data_1m('/root/daily/bit/data/1min_bitusdt.pkl')
    data_1m = data_1m.iloc[:500000]

    # Resample data to 1D
    data_1d = resample_data(data_1m, '1D')
    data_1d['Close time'] = data_1d.index
    data_1d = data_1d.reset_index(drop=True)

    # Apply Feature Calculations
    # For 1D Data
    data_1d, ma_cols_1d, ma_cols_rel_1d = calculate_MA_data(data_1d, 60, 'EMA', '_1d')
    data_1d, bb_cols_1d, bb_cols_rel_1d = calculate_ema_bollinger_bands(data_1d, 60, extra_str='_1d')
    data_1d, rsi_cols_1d = calculate_rsi(data_1d, window=20, extra_str='_1d')
    data_1d, macd_cols_1d = calculate_macd(data_1d, 20, 120, 60, extra_str='_1d')
    data_1d, stoch_cols_1d = calculate_stochastic_oscillator(data_1d, 60, 20, extra_str='_1d')
    data_1d, adx_cols_1d = calculate_adx(data_1d, 60, extra_str='_1d')
    data_1d, atr_cols_1d = calculate_atr(data_1d, 60, extra_str='_1d')
    data_1d, obv_cols_1d = calculate_obv(data_1d, extra_str='_1d')
    data_1d, will_cols_1d = calculate_williams_r(data_1d, 60, extra_str='_1d')
    data_1d, base_feature_1d = base_feature_fn(data_1d, extra_str='_1d')
    data_1d, cyclice_encoding_1d = cyclic_encode_fn(data_1d, 'Close time', 'day_of_year')

    # For 1M Data
    data_1m, ma_cols, ma_cols_rel = calculate_MA_data(data_1m, 240, 'EMA')
    data_1m, bb_cols, bb_cols_rel = calculate_ema_bollinger_bands(data_1m, 240)
    data_1m, rsi_cols = calculate_rsi(data_1m, window=60)
    data_1m, macd_cols = calculate_macd(data_1m, 60, 600, 240)
    data_1m, stoch_cols = calculate_stochastic_oscillator(data_1m, 240, 60)
    data_1m, adx_cols = calculate_adx(data_1m, 240)
    data_1m, atr_cols = calculate_atr(data_1m, 240)
    data_1m, obv_cols = calculate_obv(data_1m)
    data_1m, will_cols = calculate_williams_r(data_1m, 240)
    data_1m, base_feature = base_feature_fn(data_1m)
    data_1m, cyclice_encoding = cyclic_encode_fn(data_1m, 'Open time')

    data_1m, short_ma_cols, short_ma_cols_rel = calculate_MA_data(data_1m, 60, 'EMA')
    data_1m, long_ma_cols, long_ma_cols_rel = calculate_MA_data(data_1m, 180, 'EMA')

    # Prepare Feature Columns
    drop_column = [
        'Open time', 'Close time', 'Quote asset volume', 'Ignore',
        'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume'
    ]
    feature_column = (
        ma_cols_rel + bb_cols_rel + rsi_cols + macd_cols + stoch_cols +
        adx_cols + will_cols + base_feature + cyclice_encoding  # Excluding obv and atr
    )
    feature_column_1d = (
        ma_cols_rel_1d + bb_cols_rel_1d + rsi_cols_1d + macd_cols_1d + stoch_cols_1d +
        adx_cols_1d + will_cols_1d + base_feature_1d + cyclice_encoding_1d
    )

    # Apply Log Transform
    for feature in feature_column:
        data_1m[feature] = log_transform(data_1m[feature])

    for feature in feature_column_1d:
        data_1d[feature] = log_transform(data_1d[feature])

    data_1d['%D_20__1d'] = 0
    data_1d['ADX_60__1d'] = 0

    # bb_entry_pos_list, patience_list, bb_entry_index_list = BB_fitness_fn(data_1m)
    bb_macd_entry_pos_list, patience_list, bb_macd_entry_index_list = BB_MACD_fitness_fn(data_1m, 240, 60, 180)

    # Prepare Dataset
    data_tensor = make_dataset(
        data_1m, data_1d,
        using_column=feature_column, using_column_1d=feature_column_1d,
        window_size=240, window_size_1d=60,
        entry_pos_list=bb_macd_entry_pos_list, patience_list=patience_list,
        use_1d_data=True
    )
    entry_pos_list = np.array(bb_macd_entry_pos_list)[np.array(bb_macd_entry_pos_list) != 'hold']

    dataset_1m = []
    dataset_1d = []
    skip_data_cnt = 0
    for data in data_tensor:
        if len(data[0]) == 240 and len(data[1]) == 60:
            dataset_1m.append(torch.from_numpy(data[0]).unsqueeze(dim=0))
            dataset_1d.append(torch.from_numpy(data[1]).unsqueeze(dim=0))
        else:
            skip_data_cnt += 1
    dataset_1m = torch.cat(dataset_1m, dim=0)
    dataset_1d = torch.cat(dataset_1d, dim=0)
    dataset_1m = replace_nan_with_zero(dataset_1m)
    dataset_1d = replace_nan_with_zero(dataset_1d)

    return (data_1m, data_1d, dataset_1m, dataset_1d, skip_data_cnt, 
            entry_pos_list, bb_macd_entry_index_list, feature_column, 
            feature_column_1d)


data_1m, data_1d, dataset_1m, dataset_1d, skip_data_cnt, entry_pos_list, \
bb_macd_entry_index_list, feature_column, feature_column_1d = data_setting()

close_time_list = data_1d['Close time'].tolist()[60:-30]
dataset_rl = RLDataset(data_1m, data_1d, feature_column_1d, close_time_list, bb_macd_entry_index_list)
# dataloader_rl = DataLoader(dataset_rl, batch_size=32, shuffle=False)

def setting_model(best_index=None):
    torch.set_grad_enabled(False)
    torch.backends.cudnn.benchmark = True

    state_dict_path = '/root/daily/bit/generation/generation_39.pt'
    if os.path.exists(state_dict_path):
        state_dict = torch.load(state_dict_path)
        start_gen = state_dict['generation'] + 1
        best_profit = state_dict['best_profit']
        best_chromosomes = state_dict['best_chromosomes']

    if best_index is None:
        best_index = [i for i in range(len(best_profit))]

    chromosomes_size = len(best_index)

    # Evolution Setup
    device = 'cuda:0'
    group = len(best_index)
    prescriptor = Prescriptor(
        basic_block=None, 
        base_small_input_dim=20, 
        base_large_input_dim=20,
        base_hidden_dim=32, 
        base_output_dim=16, 
        after_input_dim=19, 
        after_hidden_dim=32, 
        after_output_dim=6, 
        num_blocks=len(best_index),
    ).to(device)

    total_param = sum(p.numel() for p in prescriptor.parameters())
    print(f"Total parameters: {total_param}")

    selection = RouletteSelection(elite_num=2000, parents_num=4000, minimize=False)
    crossover = DifferentialEvolutionOperator()
    mutation = RandomValueMutation(mut_prob=0.05)
    evolution = Evolution(
        prescriptor=prescriptor,
        selection=selection,
        crossover=crossover,
        mutation=mutation
    )

    best_chromosomes = best_chromosomes[best_index]
    init_chromosomes, base_ch_shape, after_ch_shape, device = evolution.flatten_chromosomes()
    device = 'cuda:0'
    evolution.update_chromosomes(best_chromosomes, base_ch_shape, after_ch_shape, device)

    return evolution, prescriptor, chromosomes_size, device

class CustomDataset(Dataset):
    def __init__(self, data, data_1d):
        self.data = data
        self.data_1d = data_1d

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.data_1d[idx]

def evo_inference(dataset_1m, dataset_1d, prescriptor, device):
    def inference(scaled_tensor, scaled_tensor_1d, model, device='cuda:0'):
        dataset = CustomDataset(scaled_tensor, scaled_tensor_1d)
        dataloader = DataLoader(dataset, batch_size=2048, shuffle=False, num_workers=8, pin_memory=True)
        logits = []
        for data, data_1d in dataloader:
            data = data.to(torch.float32).to(device)
            data_1d = data_1d.to(torch.float32).to(device)
            logit = model.base_forward(data, data_1d)
            logits.append(logit)
        return logits
    probs = inference(dataset_1m, dataset_1d, prescriptor, device)
    probs = torch.concat(probs, dim=1)
    probs = probs.squeeze(dim=2)

    return probs

evolution, prescriptor, chromosomes_size, device = setting_model()
rl_model = ChromosomeSelectorModel(20, 256, chromosomes_size, 4).to(device)
probs = evo_inference(dataset_1m, dataset_1d, prescriptor, device)

def train_rl(rl_model, dataset_rl, prescriptor, data_1m, probs, entry_index_list, entry_pos_list, skip_data_cnt, chromosomes_size,
             device):
    
        for data in dataset_rl:
            # rl train
            # rl action is chromosome size naming is rl_weight

            with torch.no_grad():
                # output shape: (chromosome size, 7)
                train_metrics = fitness_fn(
                    prescriptor=prescriptor,
                    data=data_1m,
                    probs=probs,
                    entry_index_list=entry_index_list,
                    entry_pos_list=entry_pos_list,
                    skip_data_cnt=skip_data_cnt,
                    start_data_cnt=skip_data_cnt,
                    chromosomes_size=chromosomes_size,
                    window_size=240,
                    alpha=1,
                    cut_percent=90,
                    device=device,
                    stop_cnt=skip_data_cnt,
                    profit_init=1,
                    limit=4
                )


                base_metric = torch.mean(train_metrics[:, 6])
                rl_metric = torch.mean(train_metrics[:, 6] * rl_weight)



100%|██████████| 500000/500000 [00:55<00:00, 8993.41it/s] 


Total parameters: 3005456


torch.Size([92, 5027, 16])